In [1]:
from keras.models import Sequential
import keras
import pandas as pd 
import numpy as np 
import lightgbm as lgb
from lightgbm.sklearn import LGBMRegressor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from Dataloader import Dataloader
from RamdomForest import RamdomForest
from score import score
from sklearn.utils import shuffle
from IPython.display import display, HTML
from datetime import date
from keras.layers import Dense
import keras.backend as K
from sklearn import preprocessing
import category_encoders as ce
%matplotlib inline

Using TensorFlow backend.


In [2]:

df = pd.read_csv('data/train.csv')
dataloader = Dataloader()

X_train, X_test, y_train, y_test = dataloader.prepare_train_data(df)



X_train_nor = dataloader.normalize(X_train)
X_test_nor = dataloader.normalize(X_test)


y_train_nor = preprocessing.scale(y_train)
y_test_nor = preprocessing.scale(y_test)

y_mean = np.mean(y_train)
y_std = np.std(y_train)

X = dataloader.prepare_train_data(df,Get_all = True)
print("saving")
X.to_csv(f'data/preprocess_x.csv',index = False)

enc_ohe = ce.one_hot.OneHotEncoder(cols=['building_material', 'city', 'town'])
df_trans = enc_ohe.fit_transform(X)
X_train_nor_dumm = enc_ohe.transform(X_train_nor)
X_test_nor_dumm = enc_ohe.transform(X_test_nor)
X_train_dumm = enc_ohe.transform(X_train)
X_test_dumm = enc_ohe.transform(X_test)

input_dim = X_train_nor_dumm.shape[1]

saving


In [3]:
test = y_train_nor*y_std + y_mean
print(test)

[ 453754.91544728  474031.22153328 1409374.32714004 ...  494990.2083452
  803593.7354361  1163691.83395193]


In [110]:
model = Sequential()
model.add(Dense(units=256, activation='relu', input_dim=input_dim))
model.add(Dense(units=8, activation='relu'))
model.add(Dense(units=1))

def custom_loss(y_true,y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true),axis=-1))

model.compile(loss=custom_loss,
              optimizer=keras.optimizers.Adam(lr=0.01))

In [111]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=100, verbose=2)

In [112]:
history = model.fit(X_train_nor_dumm, y_train_nor, epochs=20, batch_size=32,validation_data=(X_test_nor_dumm, y_test_nor))

Train on 47803 samples, validate on 11951 samples
Epoch 1/20
47803/47803 [==============================] - 8s 175us/step - loss: 0.1937 - val_loss: 0.1922
Epoch 2/20
47803/47803 [==============================] - 8s 165us/step - loss: 0.1927 - val_loss: 0.1922
Epoch 3/20
47803/47803 [==============================] - 8s 164us/step - loss: 0.1927 - val_loss: 0.1922
Epoch 4/20
47803/47803 [==============================] - 8s 164us/step - loss: 0.1927 - val_loss: 0.1922
Epoch 5/20
47803/47803 [==============================] - 8s 164us/step - loss: 0.1927 - val_loss: 0.1922
Epoch 6/20
47803/47803 [==============================] - 8s 165us/step - loss: 0.1927 - val_loss: 0.1922
Epoch 7/20
47803/47803 [==============================] - 8s 167us/step - loss: 0.1927 - val_loss: 0.1922
Epoch 8/20
47803/47803 [==============================] - 8s 166us/step - loss: 0.1927 - val_loss: 0.1922
Epoch 9/20
47803/47803 [==============================] - 8s 165us/step - loss: 0.1927 - val_loss: 0.1

KeyboardInterrupt: 

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [104]:
y = model.predict(X_train_nor_dumm, batch_size=128)
y = y*y_std + y_mean
y_true = y_train
hit_rate,point = score(y,y_true)
print(hit_rate)

first part : 999.1004748655942 last part: [0.17122656]
> 20 % : 0.8042800661046378
0.09991004748655942


In [105]:
y = model.predict(X_test_nor_dumm, batch_size=128)
y = y*y_std + y_mean
y_true = y_test

In [106]:
hit_rate,point = score(y,y_true)

first part : 978.9975734248179 last part: [0.16826367]
> 20 % : 0.809890385741779


In [107]:
hit_rate

0.0978997573424818

In [108]:
print(y_true)

34179    2.046953e+06
16800    1.056380e+06
16451    2.900460e+05
18979    4.320746e+05
10523    3.697770e+05
25780    3.219409e+05
5138     1.843707e+05
29118    1.916202e+06
52619    5.643059e+05
30268    1.767500e+05
5713     5.794772e+05
18646    5.125260e+05
41396    4.190743e+06
47537    2.513679e+06
12599    3.099014e+05
25241    5.458360e+05
25763    5.811715e+05
1562     2.971376e+05
32519    1.623881e+06
16258    2.752473e+06
48150    3.301469e+05
49100    1.390994e+06
54642    1.187174e+06
16117    2.703286e+05
36539    2.498214e+06
37091    1.417092e+05
22218    2.841575e+06
274      1.514806e+05
25732    9.479021e+04
4270     9.318327e+05
             ...     
21060    3.089881e+05
10939    3.301995e+05
53734    1.468223e+06
44474    1.297040e+06
40014    2.397404e+06
14571    2.482123e+05
53555    1.140121e+06
3175     4.331405e+05
49379    3.696249e+05
57155    7.688273e+05
28999    3.869708e+05
29996    1.765356e+06
57926    2.250594e+06
7101     1.588694e+06
39412    1

In [109]:
print(y)

[[2215828.5]
 [ 983655.9]
 [ 983655.9]
 ...
 [ 295791.3]
 [ 983655.9]
 [ 983655.9]]
